# **SQL PROGRAMMING**

You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state. You are required:
1. To join the 2 tables
2. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that
3. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
5. To compute the running total of Grand Electors in that sorted list.
6. Independently, to compute the half of the total of Grand Electors overall (in the whole country).
7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.

# **Lets connect to our Database**

In [ ]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed.
%sql sqlite://

'Connected: @None'

In [ ]:
# Importing Pandas
import pandas as pd

# **Loading our Datasets**

Datasets used in this notebook:

1. **Grand electors by State.**
2. **Population by State.**


In [ ]:
# Let's load our Grand Electors by state dataset below
#
with open('GrandElectors_by_state.csv','r') as f:
    GrandElectors_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS GrandElectors_by_state;
%sql PERSIST GrandElectors_by_state;
%sql SELECT * FROM GrandElectors_by_state LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [ ]:
# Let's load our Cities dataset below
#
with open('Population_by_state.csv','r') as f:
    Population_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Population_by_state;
%sql PERSIST Population_by_state;
%sql SELECT * FROM Population_by_state LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


# **Question 1. Joining the 2 tables**

In [ ]:
# To join the Grand electors and population datasets,
# we will need to convert the grand elector's state colum values
# to uppercase as below.
%%sql
UPDATE
  GrandElectors_by_state
SET
  State = UPPER(State)

 * sqlite://
51 rows affected.


[]

In [ ]:
# After ensuring the state column which is the commob one
# between the two tables, we can now proceed and join the tables
# as here under
%%sql
SELECT GrandElectors_by_state.State, GrandElectors, Population
FROM GrandElectors_by_state INNER JOIN Population_by_state
     ON GrandElectors_by_state.State = Population_by_state.State
ORDER BY Population DESC
LIMIT 10

 * sqlite://
Done.


State,GrandElectors,Population
CALIFORNIA,55,39144818
TEXAS,38,27469114
FLORIDA,29,20271272
NEW YORK,29,19795791
ILLINOIS,20,12859995
PENNSYLVANIA,20,12802503
OHIO,18,11613423
GEORGIA,16,10214860
NORTH CAROLINA,15,10042802
MICHIGAN,16,9922576


# **Question Two: Changing District of Colombia to DC**

In [ ]:
# We will change the name of of the "District of Columbia" in the Grand Elector by state dataset
# to its short version "DC"
%%sql
UPDATE GrandElectors_by_state
SET
    State = 'DC'
WHERE
    State = 'DISTRICT OF COLUMBIA';

 * sqlite://
1 rows affected.


[]

In [ ]:
# We will change the name of of the "District of Columbia" in the Population by state dataset
# to its short version "DC"
%%sql
UPDATE Population_by_state
SET
    State = 'DC'
WHERE
    State = 'DISTRICT OF COLUMBIA';

 * sqlite://
1 rows affected.


[]

In [ ]:
# Lets now create a new table after joining and renaming District of Columbia
# this will make our work easier
%%sql
CREATE TABLE US_Voters_data
(
  State varchar(255),
  Population float,
  GrandElectors float
);

INSERT INTO US_Voters_data(State, Population, GrandElectors)
SELECT GrandElectors_by_state.State, Population_by_state.Population, GrandElectors_by_state.GrandElectors
FROM GrandElectors_by_state
INNER JOIN Population_by_state on GrandElectors_by_state.State = Population_by_state.State


 * sqlite://
Done.
51 rows affected.


[]

# **Question Three: Compute the ratio between the number of grand electors and the population**

In [ ]:
# Now we will calcualte the ratio between grand electors and population for each state
# as below
%%sql
SELECT * , (Population)/(GrandElectors) as Ratio FROM US_Voters_data

 * sqlite://
Done.


State,Population,GrandElectors,Ratio
ALABAMA,4858979.0,9.0,539886.5555555555
ALASKA,738432.0,3.0,246144.0
ARIZONA,6828065.0,11.0,620733.1818181818
ARKANSAS,2978204.0,6.0,496367.3333333333
CALIFORNIA,39144818.0,55.0,711723.9636363636
COLORADO,5456574.0,9.0,606286.0
CONNECTICUT,3590886.0,7.0,512983.71428571426
DC,672228.0,3.0,224076.0
DELAWARE,945934.0,3.0,315311.3333333333
FLORIDA,20271272.0,29.0,699009.3793103448


# **Question 4: Order the states by decreasing ratio of Grand Electors per capita**

In [ ]:
# After calculating the grand electors per capita,
# we will now order the data in descending order and have our priority list of
# at least the top 20 states
# as here under
%%sql
SELECT * , (Population)/(GrandElectors) as Ratio FROM US_Voters_data
ORDER BY Ratio DESC
LIMIT 20

 * sqlite://
Done.


State,Population,GrandElectors,Ratio
TEXAS,27469114.0,38.0,722871.4210526316
CALIFORNIA,39144818.0,55.0,711723.9636363636
FLORIDA,20271272.0,29.0,699009.3793103448
NEW YORK,19795791.0,29.0,682613.4827586206
NORTH CAROLINA,10042802.0,15.0,669520.1333333333
OHIO,11613423.0,18.0,645190.1666666666
VIRGINIA,8382993.0,13.0,644845.6153846154
ILLINOIS,12859995.0,20.0,642999.75
PENNSYLVANIA,12802503.0,20.0,640125.15
NEW JERSEY,8958013.0,14.0,639858.0714285715


# **Question Five: Compute the running total of Grand Electors in that sorted list**

In [ ]:
# Now that we have sorted our data, we will compute the running total
# of the grand electors in the soreted list as below
%%sql
SELECT State, GrandElectors, (Population)/(GrandElectors) as Ratio,
SUM(GrandElectors) OVER (ORDER BY State) AS RunningBalance
FROM US_Voters_data
ORDER BY Ratio DESC
LIMIT 20

 * sqlite://
Done.


State,GrandElectors,Ratio,RunningBalance
TEXAS,38,722871.4211,38
CALIFORNIA,55,711723.9636,93
FLORIDA,29,699009.3793,122
NEW YORK,29,682613.4828,151
NORTH CAROLINA,15,669520.1333,166
OHIO,18,645190.1667,184
VIRGINIA,13,644845.6154,197
ILLINOIS,20,642999.75,217
PENNSYLVANIA,20,640125.15,237
NEW JERSEY,14,639858.0714,251


# **Question Six: Independently, to compute the half of the total of Grand Electors overall**

In [ ]:
# Having determined the total number of grand electors
# we will now find the half of the total grand electors that will
# guarantee our boss the win.
%%sql
SELECT SUM(GrandElectors)/2 , (Population)/(GrandElectors) as Ratio FROM US_Voters_data

 * sqlite://
Done.


SUM(GrandElectors)/2,Ratio
269.0,195369.0


# **Question Seven: Filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold.**

In [ ]:
# Now that we have determined that to win we need to garner 270 electors votes
# or at least 269 electors votes, we will filter our list and drop non-priority states
%%sql
SELECT SUM(GrandElectors) From US_Voters_data
WHERE GrandElectors > 13


 * sqlite://
Done.


SUM(GrandElectors)
270.0


In [ ]:
# Lets list the states that meet the above threshhold here below
# The list display here under will conatin our target list
# If we can convince the voter in these states, then our win is guaranteed.
%%sql
SELECT State, GrandElectors, Population FROM US_Voters_data
WHERE GrandElectors > 13
ORDER BY GrandElectors DESC

 * sqlite://
Done.


State,GrandElectors,Population
CALIFORNIA,55.0,39144818.0
TEXAS,38.0,27469114.0
FLORIDA,29.0,20271272.0
NEW YORK,29.0,19795791.0
ILLINOIS,20.0,12859995.0
PENNSYLVANIA,20.0,12802503.0
OHIO,18.0,11613423.0
GEORGIA,16.0,10214860.0
MICHIGAN,16.0,9922576.0
NORTH CAROLINA,15.0,10042802.0


**THE END**